In [ ]:
## From: https://levelup.gitconnected.com/jira-api-with-python-and-pandas-c1226fd41219
## Sample code: https://gist.github.com/dmort-ca/a3bbeaac4729ba2c72a9a33512402ae4

In [ ]:
from atlassian import Jira
import pandas as pd
import matplotlib.pyplot as plt

## From: https://id.atlassian.com/manage-profile/security/api-tokens
  * API Tokens

In [ ]:
keyfile = open("apikey.txt")

apikey = keyfile.read().replace("\n","")
keyfile.close()

In [ ]:
INSTANCE_URL = "https://HOSTNAME.atlassian.net/"
USER_NAME="EMAIL@COMPANY.com"

## https://atlassian-python-api.readthedocs.io/

In [ ]:
jira_instance = Jira(
    url=INSTANCE_URL,
    username=USER_NAME,
    password=apikey,
    cloud=True)

In [ ]:
jira_instance

In [ ]:
jira_instance.issue(key = "WEB-23", fields = ["issuetype", "status", "summary"])

In [ ]:
def retrieve_all_query_results(jira_instance: Jira, query_string: str, fields: list) -> list:
    issues_per_query = 5
    list_of_jira_issues = []
    
    # Get the total issues in the results set. This is one extra request but it keeps things simple.
    num_issues_in_query_result_set = jira_instance.jql(query_string, limit = 0)["total"]
    print(f"Query `{query_string}` returns {num_issues_in_query_result_set} issues")
    
    # Use floor division + 1 to calculate the number of requests needed
    for query_number in range(0, (num_issues_in_query_result_set // issues_per_query) + 1):
        results = jira_instance.jql(query_string, limit = issues_per_query, start = query_number * issues_per_query, fields = fields)
        print (query_number, query_number * issues_per_query)
        list_of_jira_issues.extend(results["issues"])
    
    return list_of_jira_issues

In [ ]:
first_query_results = retrieve_all_query_results(jira_instance, "Project = WEB", fields = ["issuelinks", "summary"])
# Create an empty list to add issues when not all the blocking issues are closed.

In [ ]:
df = pd.json_normalize(first_query_results)
df.head()

In [ ]:
FIELDS_OF_INTEREST = ["key", "fields.summary", "fields.summary", "fields.issuelinks"]
df = pd.json_normalize(first_query_results)
df[FIELDS_OF_INTEREST].head()

In [ ]:
df.columns

In [ ]:
jql_result_set_issues = retrieve_all_query_results(jira_instance, "Project = WEB", fields = "*all")
df = pd.json_normalize(jql_result_set_issues)
df.head()

In [ ]:
jql_result_set_issues[0]['fields']

In [ ]:
all_projects = jira_instance.projects(included_archived=None)
projects_df = pd.json_normalize(all_projects)


In [ ]:
PROJ_FIELDS_OF_INTEREST = ["key", "name", "id", "projectTypeKey"]
projects_df[PROJ_FIELDS_OF_INTEREST].head()

In [ ]:
account_id = '111111111111111111111111'
u = jira_instance.get_user_groups(account_id)
u

In [ ]:
# Get all users from group
group= 'site-admins'
g = jira_instance.get_all_users_from_group(group, include_inactive_users=False, start=0, limit=50)
g

In [ ]:
group_df = pd.json_normalize(g['values'])
GROUP_FIELDS_OF_INTEREST = [ "displayName", "emailAddress",  "self", "accountId", "active"]
group_df[GROUP_FIELDS_OF_INTEREST].head()


## Requests Library
  * https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-user-search/#api-rest-api-3-user-search-query-get
  * search

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

url = INSTANCE_URL + "/rest/api/2/user"

auth = HTTPBasicAuth(USER_NAME, apikey)

headers = {
  "Accept": "application/json"
}


account_id = '111111111111111111111111'
query = {
  'accountId': account_id
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   params=query,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


In [ ]:
url = INSTANCE_URL + "/rest/api/3/user/search"

auth = HTTPBasicAuth(USER_NAME, apikey)

headers = {
  "Accept": "application/json"
}

# Syntax: https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-user-search/#api-rest-api-3-user-search-get

query = {
  'query': 'query=k'
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   params=query,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


In [ ]:
url = INSTANCE_URL + "/rest/api/3/user/search/query"

auth = HTTPBasicAuth(USER_NAME, apikey)

headers = {
  "Accept": "application/json"
}

# Syntax: https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-user-search/#api-rest-api-3-user-search-get

query = {
  'query': 'is assignee of WEB'
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   params=query,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


In [ ]:
url = INSTANCE_URL + "/rest/api/3/user/viewissue/search"

auth = HTTPBasicAuth(USER_NAME, apikey)

headers = {
  "Accept": "application/json"
}

query = {
  'query': 'query=kev',
  'projectKey': 'WEB'
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   params=query,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

## User ID's
  * https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-users/#api-rest-api-3-user-bulk-migration-get

In [ ]:
## https://levelup.gitconnected.com/jira-api-with-python-and-pandas-c1226fd41219

In [ ]:
url = INSTANCE_URL + "/rest/api/3/users/search"

auth = HTTPBasicAuth(USER_NAME, apikey)

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

In [ ]:
j = response.json()
u_df = pd.DataFrame.from_dict(j)
rslt_df = u_df[u_df['accountType'] == 'atlassian'] 
USER_FIELDS_OF_INTEREST = [ "displayName", "emailAddress",  "self", "accountId", "active"]
rslt_df[USER_FIELDS_OF_INTEREST].head()

# Done - ZZZZ

## Custom Fields

In [ ]:
custom_fields_list = jira_instance.get_all_custom_fields()
df_custom_fields = pd.DataFrame.from_records(custom_fields_list, index = ["key"])

In [ ]:
## https://gist.github.com/dmort-ca/a3bbeaac4729ba2c72a9a33512402ae4